# Quickstart

In this notebook, we'll introduce the API that defines entities species, genes, and proteins.

It shows how to 

- query different semantic representations based on underlying ontologies
- standardize columns in `DataFrame` objects

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bionty as bt
import pandas as pd

## Species

In [ ]:
species = bt.Species(common_name="human")

Dataclass allows users to access each species entry via tab completion

In [ ]:
model = species.dataclass

In [ ]:
model.mouse

Listing species fields

In [ ]:
species.fields

The field of standardized id

In [ ]:
species.std_id

The value of the standardized id

In [ ]:
species.std_name

Searching for a field

In [ ]:
species.search("scientific_name"), species.search("assembly")

## Gene

In [ ]:
gn = bt.Gene(species="human")

Listing gene related fields

In [ ]:
gn.fields

The field of standardized id

In [ ]:
gn.std_id

Dataclass allows users to access each gene entry via tab completion

In [ ]:
model = gn.dataclass

In [ ]:
model.PDCD1

In [ ]:
model.PDCD1.ensembl_gene_id

Mapping between fields

In [ ]:
hgnc_ids = ["HGNC:1100", "HGNC:1101"]
ensembl_ids = ["ENSG00000012048", "ENSG00000139618"]

In [ ]:
# default is to convert into .std_id

gn.search(ensembl_ids, id_type_from="ensembl.gene_id")

In [ ]:
# OR you can convert between any two of the attributes

gn.search(["BRCA1", "BRCA2"], id_type_from="hgnc_symbol", id_type_to="entrez.gene_id")

Standardizing gene symbols

In [ ]:
# default is to standardizing gene symbols

df = pd.DataFrame(index=["RNF53", "BRCA2", "FakeGene"])
gn.standardize(df)

df

In [ ]:
# can also input e.g. ensembl id

df = pd.DataFrame(index=["ENSG00000012048", "ENSG00000139618"])
gn.standardize(df, id_type="ensembl.gene_id")

df

## Protein

In [ ]:
pt = bt.Protein(species="human")

In [ ]:
pt.fields

In [ ]:
pt.std_id

In [ ]:
uniprot_ids = ["P40925", "P40926", "O43175", "Q9UM73"]

pt.search(uniprot_ids, id_type_from="UNIPROT_ID", id_type_to="CHEMBL_ID")

## CellType

In [ ]:
ct = bt.CellType()

In [ ]:
ct.onto

`.onto_dict` gives you the {name:label}

In [ ]:
ct.onto_dict["CL_0002000"]

You can also access to the individule objects in .`classes`

In [ ]:
obj = ct.classes["CL_0002000"]

In [ ]:
obj.is_a

In [ ]:
obj.ancestors()

Let's try to search for some CL terms

In [ ]:
ct.search(["T cell", "B cell", "hepatocyte"])

Standardize multiple terms

In [ ]:
terms = ["CL:0000084", "CL:0000243", "CL:0002000"]

ct.standardize(terms)